# Solución María

### 0. Importaciones

In [1]:
import pandas as pd
import sqlite3

conn = sqlite3.connect('./Data/sql-murder-mystery.db')
cursor_ch = conn.cursor()
sql_query = lambda query: pd.read_sql(query, conn)

pd.set_option('display.max_colwidth', None)

### 1. Datos iniciales

In [2]:
query = '''
SELECT * FROM crime_scene_report
WHERE date = 20180115
AND type = 'murder'
AND city = 'SQL City'
'''

sql_query(query)

,date,type,description,city
0,20180115,murder,"Security footage shows that there were 2 witnesses. The first witness lives at the last house on ""Northwestern Dr"". The second witness, named Annabel, lives somewhere on ""Franklin Ave"".",SQL City


### 2. Búsqueda de testigos y acceso a sus interrogatorios

In [3]:
query = '''
SELECT 
	A.id, 
	A.name,
	A.address_street_name,
	A.address_number,
	B.transcript
 
FROM person A
JOIN interview B
ON A.id = B.person_id

WHERE address_street_name ='Northwestern Dr' -- Testigo 1: vive en la última casa (MAX()) de la calle Northwestern Dr
AND address_number = (SELECT MAX(address_number) FROM person WHERE address_street_name ='Northwestern Dr')
OR address_street_name = 'Franklin Ave' -- Testigo 2: viven en la calle Franklin Ave y su nombre es 'Annabel', no sabemos su apellido
AND name LIKE 'Annabel%'
'''
sql_query(query)

,id,name,address_street_name,address_number,transcript
0,14887,Morty Schapiro,Northwestern Dr,4919,"I heard a gunshot and then saw a man run out. He had a ""Get Fit Now Gym"" bag. The membership number on the bag started with ""48Z"". Only gold members have those bags. The man got into a car with a plate that included ""H42W""."
1,16371,Annabel Miller,Franklin Ave,103,"I saw the murder happen, and I recognized the killer from my gym when I was working out last week on January the 9th."


### 3. Encontrar al asesino

In [4]:
query = '''
SELECT 
	A.id, 
 	A.name, 
  	A.license_id, 
	B.plate_number,
	C.id,
	D.check_in_date,
	D.check_in_time,
	D.check_out_time
 
FROM person A
JOIN drivers_license B ON A.license_id = B.id
JOIN get_fit_now_member C ON A.id = C.person_id
JOIN get_fit_now_check_in D ON C.id = D.membership_id

WHERE plate_number LIKE '%H42W%' -- Tiene una matrícula como la que dice Morty
AND membership_id LIKE '48Z%' -- Tiene suscripción al gimnasio que empieza por 48Z
AND membership_status = 'gold' -- Miembro gold
OR person_id = 16371 -- Coincide con Annabel en el gimnasio
'''

sql_query(query)

,id,name,license_id,plate_number,id,check_in_date,check_in_time,check_out_time
0,67318,Jeremy Bowers,423327,0H42W2,48Z55,20180109,1530,1700
1,16371,Annabel Miller,490173,23AM98,90081,20180109,1600,1700


### 4. Interrogatorio del primer asesino e información sobre el verdadero cerebro

In [5]:
query = '''
SELECT * FROM interview
WHERE person_id = 67318
'''
sql_query(query)

,person_id,transcript
0,67318,"I was hired by a woman with a lot of money. I don't know her name but I know she's around 5'5"" (65"") or 5'7"" (67""). She has red hair and she drives a Tesla Model S. I know that she attended the SQL Symphony Concert 3 times in December 2017.\n"


### 5. Identificación del verdadero asesino

In [6]:
query = '''
SELECT 
	A.id,
 	A.name,
	B.height,
	B.hair_color,
	B.car_make,
	B.car_model,
	C.annual_income,
	D.event_name,
	D.date
 
FROM person A
JOIN drivers_license B ON A.license_id = B.id
JOIN income C ON A.ssn = C.ssn
JOIN facebook_event_checkin D ON A.id = D.person_id

WHERE B.gender = 'female'
AND height BETWEEN 65 AND 67 -- Altura entre 5'5" y 5'6"
AND hair_color = 'red' -- Pelirroja
AND car_make = 'Tesla' -- Marca del coche
AND car_model = 'Model S' -- Modelo del coche
AND date BETWEEN 20171201 AND 20171231 -- Asiste 3 veces al SQL Symphony Concert en diciembre 2017
AND event_name = 'SQL Symphony Concert'
'''

df = sql_query(query)
df['date'] = pd.to_datetime(df.date.astype(str), format = '%Y%m%d')
df

,id,name,height,hair_color,car_make,car_model,annual_income,event_name,date
0,99716,Miranda Priestly,66,red,Tesla,Model S,310000,SQL Symphony Concert,2017-12-06
1,99716,Miranda Priestly,66,red,Tesla,Model S,310000,SQL Symphony Concert,2017-12-12
2,99716,Miranda Priestly,66,red,Tesla,Model S,310000,SQL Symphony Concert,2017-12-29


In [23]:
query = '''
SELECT A.name, A.id, A.license_id, B.car_make, B.car_model
FROM person A
JOIN drivers_license B ON A.license_id = B.id
'''

df = sql_query(query)

print(df[df['car_make'].isin(['Tesla']) == True], '\n')
print(df[df['car_model'].isin(['Model S']) == True], '\n')

# Pero creo que lo más facil para ver si coinciden los dos es una máscara
df[(df.car_make == 'Tesla') & (df.car_model == 'Model S')]

                  name     id  license_id car_make car_model
310      Dion Montague  13064      467669    Tesla   Model S
1178      Clair Koback  20871      157563    Tesla  Roadster
1708   Oralia Fleisher  25788      544252    Tesla  Roadster
2475      Mei Bianchin  32858      405170    Tesla   Model 3
3109  Kirsten Paetzold  38459      688398    Tesla  Roadster
4544      Sharell Boyn  50769      909986    Tesla   Model X
5271    Cletus Zoeller  57410      736081    Tesla   Model S
6365    Alyssa Faggett  67673      682231    Tesla   Model S
7238   Spring Federick  75162      595803    Tesla  Roadster
7660          Red Korb  78881      918773    Tesla   Model S
7853  Juliet Calderara  80666      513007    Tesla  Roadster
8972     Regina George  90700      291182    Tesla   Model S
9363         Jon Haldi  94330      638311    Tesla   Model S
9980  Miranda Priestly  99716      202298    Tesla   Model S 

                  name     id  license_id car_make car_model
310      Dion Montague

,name,id,license_id,car_make,car_model
310,Dion Montague,13064,467669,Tesla,Model S
5271,Cletus Zoeller,57410,736081,Tesla,Model S
6365,Alyssa Faggett,67673,682231,Tesla,Model S
7660,Red Korb,78881,918773,Tesla,Model S
8972,Regina George,90700,291182,Tesla,Model S
9363,Jon Haldi,94330,638311,Tesla,Model S
9980,Miranda Priestly,99716,202298,Tesla,Model S
